# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator # 이미지 전처리를 위한 모듈

Mounted at /content/drive


## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,  # featureScaling
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
#  image argumentation : 과적합을 방지하기 위해 train set에만 적용하고 test set에는 적용하지 않는다.
#  이미지에 기하학적 변화를 주거나 확대/축소하고 회전을 시킨다.
#  training set의 이미지를 변화시켜 overfit이 발생하는 것을 방지한다.
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/dataset/training_set',
                                                 target_size = (64, 64),  # 이미지의 최종 크기 (크면 training 시간이 길어진다.)
                                                 batch_size = 32, # 각 배치에 몇개의 이미지를 사용할 것인지? 보통 32로 한다.
                                                 class_mode = 'binary')  # binary or categorical 여기서는 개, 고양이만 있으니까 binary

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [ ]:
cnn = tf.keras.models.Sequential()

### Convolution Layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))  
# filters  특징감지기의 갯수
# kernel_size = 3*3 커널 사용
# target_size [64,64] 와 color image 니까 3을 입력.(흑백이면 1)

### Pooling Layer

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# pooling frame size 2*2
# 보폭 2

### Adding additional Layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten())
#  1차원 벡터로 펼치기.

### Step 4 - Full Connection

In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dropout(rate=0.2))
#  units : 히든 뉴런의 갯수, 뉴런이 많으면 정확도가 높아질 것이다.
#  output layer가 아니기 때문에. -> 'relu' 사용

### Step 5 - Output Layer

In [ ]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
#  output은 여기서는 이진분류이므로 개, 고양이 0,1만 있으면 되니까 units = 1
#  output은 sigmoid (이진분류), softmax(다중분류)

## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#  optimizer : adam
#  loss : 
#  metrics : 다중 선택을 할 수 있지만 지금은 accuracy만 사용.

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 5289s 21s/step - loss: 0.6767 - accuracy: 0.5934 - val_loss: 0.6457 - val_accuracy: 0.6560
Epoch 2/25
250/250 [==============================] - 79s 318ms/step - loss: 0.6264 - accuracy: 0.6565 - val_loss: 0.5908 - val_accuracy: 0.6925
Epoch 3/25
250/250 [==============================] - 81s 323ms/step - loss: 0.5806 - accuracy: 0.6945 - val_loss: 0.5496 - val_accuracy: 0.7320
Epoch 4/25
250/250 [==============================] - 81s 323ms/step - loss: 0.5502 - accuracy: 0.7170 - val_loss: 0.5171 - val_accuracy: 0.7540
Epoch 5/25
250/250 [==============================] - 81s 322ms/step - loss: 0.5187 - accuracy: 0.7383 - val_loss: 0.5087 - val_accuracy: 0.7515
Epoch 6/25
250/250 [==============================] - 79s 316ms/step - loss: 0.4901 - accuracy: 0.7607 - val_loss: 0.4910 - val_accuracy: 0.7690
Epoch 7/25
250/250 [==============================] - 81s 324ms/step - loss: 0.4676 - accuracy: 0.7745 - val_loss: 0.4926 - val_ac

## Part 4 - Making a single prediction

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/content/drive/MyDrive/dataset/single_prediction/cat_or_dog_2.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)  # PIL 형식의 이미지를 배열로 바꿔준다.
test_image = np.expand_dims(test_image, axis = 0)  
# 우리가 만든 모델은 배치사이즈가 32이다. 그래서 예측할때도 같은 배치로 해줘야한다.
# 따라서 추가 배치를 해준다.
result = cnn.predict(test_image/255.0)
print(training_set.class_indices)
if result[0][0] > 0.5:
  prediction = 'dog'
else:
  prediction = 'cat'

print(prediction)

{'cats': 0, 'dogs': 1}
cat
